## **BDSN END TERM ASSIGNMENT**

Installing the latest Version of Spark


In [1]:
!apt update > /dev/null
!apt install openjdk-8-jdk-headless -qq > /dev/null



^C




In [2]:
# Get latest and correct version of Spark
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 56.1 MB/s 


Importing SparkSession from Pyspark and building SparkSession


In [3]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()

Importing KMeans from ml.clustering Module of Pyspark

In [4]:
from pyspark.ml.clustering import KMeans

Reading the dataset using read.csv function of Pyspark with Header as True and inferSchema as true to check the structure of the dataset

In [5]:
dataset=spark.read.csv("/content/hack_data.csv",header=True,inferSchema=True)

Counting the number of Rows from the datasets

In [6]:
dataset.count()

334

Using PrintSchema() function to check the strucure of the data

In [7]:
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



Printing the first 5 rows of the data

In [59]:
dataset.head(5)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08),
 Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58),
 Row(Session_Connection_Time=2.0, Bytes Transferred=228.08, Kali_Trace_Used=1, Servers_Corrupted=2.48, Pages_Corrupted=8.0, Location='Bolivia', WPM_Typing_Speed=70.8),
 Row(Session_Connection_Time=20.0, Bytes Transferred=408.5, Kali_Trace_Used=0, Servers_Corrupted=3.57, Pages_Corrupted=8.0, Location='Iraq', WPM_Typing_Speed=71.28)]

We Use describe().show() function to obtain the statistical summaries of the data which shows us the mean Time of the whole operation by the hackers is 30 minutes with SD being 14 minutes which means either one attack is 44 minutes and the other is 16 minutes corroborating 60 minutes as maximum time. Assuming there are 2 clusters
The Average clusters destroyed is 5 and average pages corrupted is 11 and WPM  typing speed ona an average is 57 minutes




In [7]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

We then import VectorAssembler which is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees.It can also be used for Clustering

In [8]:
from pyspark.ml.feature import VectorAssembler

checking for columns before applying transformations

In [9]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

AS CLUSTERING IS UNSUPERVISED LEARNING WE DO NOT NEED LOCATION DATA AS IT IS A LABEL. THERE ARE NO LABELS IN UNSUPERVISED LEARNING. ALSO HACKERS USUALLY USE VPNs TO HIDE THEIR LOCATION SO ITS NOT THAT USEFUL SO WE DONT CONSIDER IT.

In [11]:
feature_columns=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

We then apply Transformations to columns after removing the locations

In [13]:
assembler=VectorAssembler(inputCols=feature_columns,outputCol="features")

TRANSFORMING THE DATA


In [14]:
final_data=assembler.transform(dataset)

We check for structure of the data to see the new structure of the dataset

In [15]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



We import StandardScaler to standardize the data

In [16]:
from pyspark.ml.feature import StandardScaler

We standardize the features and name the new features as featuresScaled

In [19]:
scale=StandardScaler(inputCol="features",outputCol="featuresScaled")

We then fit the scaled data and apply transformation as store it as a cluster_final_data 

In [20]:
scaled_model=scale.fit(final_data)
cluster_final_data=scaled_model.transform(final_data)

We print the schema of cluster_final_data

In [21]:
cluster_final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- featuresScaled: vector (nullable = true)



We compute the cost of clusters for different values of K so that optimum value should be selected

We compute the cost for K=2 below




In [29]:

kmeans = KMeans().setK(2).setSeed(1).setFeaturesCol('featuresScaled')
model = kmeans.fit(cluster_final_data)
cost_cluster = model.summary.trainingCost
print("The cost of clusters are = " + str(cost_cluster))

The cost of clusters are = 601.7707512676691


We compute the cost for K=3 below


In [30]:
kmeans = KMeans().setK(3).setSeed(1).setFeaturesCol('featuresScaled')
model = kmeans.fit(cluster_final_data)
cost_cluster = model.summary.trainingCost
print("The cost of clusters are = " + str(cost_cluster))

The cost of clusters are = 434.75507308487596


We create a variable Kmeans3 and Apply KMeans function to scaled features and take K as 3 

In [49]:
kmeans3= KMeans(featuresCol="featuresScaled",k=3)

We fit Kmeans3 to final data and this our model for K=3

In [50]:
model_k3=kmeans3.fit(cluster_final_data)

We then see the prediction for K=3 and find that it is not evenly traded off as per the domain experts information provided to us 

In [52]:
model_k3.transform(cluster_final_data).groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|   88|
|         0|  167|
+----------+-----+



Now we create a model for K=2 and find that its evely split so we conclude that there are 2 hackers

In [53]:
kmeans2= KMeans(featuresCol="featuresScaled",k=2)

In [54]:
model_k2=kmeans2.fit(cluster_final_data)

In [58]:
model_k2.transform(cluster_final_data).groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

